<a href="https://colab.research.google.com/github/gio961gio/Audio-Clustering-with-Python-and-Max-MSP/blob/main/Audio_Clustering_with_Python_and_Max_MSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title **Cloning Github**
import sys
!git clone https://github.com/gio961gio/Audio-Clustering-with-Python-and-Max-MSP.git
sys.path.append("/content/Audio-Clustering-with-Python-and-Max-MSP")

In [ ]:
# @title **Setup Load Dataset (Google Drive Link)**
!pip install rarfile
!pip install pydub


import os
import gdown
import rarfile

# Definisci il link condiviso al file RAR su Google Drive
rar_file_link = "https://drive.google.com/file/d/1QBaEbp3psMadVr3cgdACvT_43bka7tvO/view?usp=drive_link" # @param {type:"string"}

# Estrai l'ID del file RAR dal link condiviso
file_id = rar_file_link.split('/')[-2]

# Scarica il file RAR da Google Drive
rar_file_path = "/content/750_audio.rar" # @param {type:"string"}
gdown.download(f'https://drive.google.com/uc?id={file_id}', rar_file_path, quiet=False)


# Specifica la directory di destinazione dell'estrazione
extraction_path = "/content/extracted_files/"  # @param {type:"string"}

# Crea la directory di destinazione se non esiste
os.makedirs(extraction_path, exist_ok=True)

# Esegui l'estrazione dei file RAR
with rarfile.RarFile(rar_file_path, 'r') as rar:
    rar.extractall(extraction_path)

In [ ]:
# @title **Audio Features Extractor**
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
import warnings
from audio_stuff import extract_audio_features_from_directory


# Ignora i warning
warnings.filterwarnings("ignore")

def conta_file_in_cartella(cartella):
    # Ottieni la lista dei file nella cartella
    lista_file = os.listdir(cartella)

    # Conta quanti elementi nella lista sono file (escludendo le directory)
    num_file = sum(os.path.isfile(os.path.join(cartella, f)) for f in lista_file)

    return num_file


# Ottieni il numero di file nella cartella specificata
num_file = conta_file_in_cartella(extraction_path)



num_samples = 10 # @param {type:"integer"}

if num_samples < 1 or num_samples > num_file:
    raise ValueError(f"Number of samples must be between 1 and {num_file}.")



# Ottieni la lista dei file audio estratti
audio_files = [os.path.join(extraction_path, filename) for filename in os.listdir(extraction_path) if filename.endswith('.wav')][:]
audio_files = sorted(audio_files, key=lambda x: int(x.split('/')[-1].split('_')[0]))
audio_files_ = audio_files[:num_samples]



combined_features = extract_audio_features_from_directory('/content/extracted_files', num_mfcc=13, num_samples=num_samples)


# Trasforma la lista di caratteristiche in un array numpy
features_array = np.array(combined_features)

# Feature Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_array)


# Salva scaled_features in un file
np.save('scaled_features.npy', scaled_features)


In [ ]:
# @title **Clustering 2D**
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np

#Parametri
n_clusters = 4 # @param {type:"integer"}
init='k-means++'  # @param ["k-means++", "random"]
n_init=100 # @param {type:"integer"}
max_iter=10 # @param {type:"integer"}




# Carica i dati da file
scaled_features = np.load('scaled_features.npy')


# Crea il modello PCA con 2 componenti principali
pca = PCA(n_components=2)

# Applica la PCA alle feature scalate (scaled_features)
reduced_features = pca.fit_transform(scaled_features)

# # Riscala i risultati dell'PCA tra 0 e 1
scaler = MinMaxScaler()
reduced_features = scaler.fit_transform(reduced_features)

# Esegui il clustering sulle feature ridotte
minibatch_kmeans = MiniBatchKMeans(n_clusters=n_clusters, init=init, n_init=n_init, max_iter=max_iter)
cluster_labels = minibatch_kmeans.fit_predict(reduced_features)

# Calcola il Silhouette Score per valutare la bontà del clustering
silhouette_avg = silhouette_score(reduced_features, cluster_labels)
print(f"Silhouette Score: {silhouette_avg}")

# # Calcola l'inerzia (inertia) per valutare la compattazione dei cluster
# inertia = minibatch_kmeans.inertia_
# print(f"Inertia (Inerzia): {inertia}")


# Visualizza i risultati del clustering
plt.figure(figsize=(10, 6))
unique_labels = np.unique(cluster_labels)
for label in unique_labels:
    cluster_points = reduced_features[cluster_labels == label]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {label + 1}')

plt.xlabel('PC1')
plt.ylabel('PC2')

plt.title('Risultato del Clustering dei Loop Audio con MiniBatchKMeans e PCA')
plt.legend()

plt.show()


# Save/Load Model (Optional)

In [ ]:
# @title Save Model
import joblib  # Per il salvataggio e il caricamento del modello
model_name = 'modelD.pkl' # @param {type:"string"}
joblib.dump(minibatch_kmeans, model_name)

In [ ]:
# @title Load Model
import joblib

model_name = 'modelD.pkl' # @param {type:"string"}

minibatch_kmeans = joblib.load(model_name)
cluster_labels = minibatch_kmeans.fit_predict(reduced_features)

# # Calcola il Silhouette Score per valutare la bontà del clustering
# silhouette_avg = silhouette_score(reduced_features, cluster_labels)
# print(f"Silhouette Score: {silhouette_avg}")

# # Calcola l'inerzia (inertia) per valutare la compattazione dei cluster
# inertia = minibatch_kmeans.inertia_
# print(f"Inertia (Inerzia): {inertia}")


# Visualizza i risultati del clustering
plt.figure(figsize=(10, 6))
unique_labels = np.unique(cluster_labels)
for label in unique_labels:
    cluster_points = reduced_features[cluster_labels == label]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {label + 1}')

plt.xlabel('PC1')
plt.ylabel('PC2')

plt.title('Risultato del Clustering dei Loop Audio con MiniBatchKMeans e PCA')
plt.legend()

plt.show()

# Files for Max MSP

In [ ]:
# @title IMUBU

import string

letters = string.ascii_uppercase
cluster_to_letter = {}

# Mappa i cluster a lettere maiuscole
for x in range(n_clusters):
    cluster_to_letter[x] = letters[x]

# Crea una lista di coordinate, numero di cluster e lettere maiuscole corrispondenti
results = []

for label in unique_labels:
    cluster_points = reduced_features[cluster_labels == label]
    cluster_letter = cluster_to_letter[label]
    cluster_num = label + 1

    for point in cluster_points:
        x, y = point[:2]  # Prendi solo le prime due coordinate PC1 e PC2
        x = round(x, 4)  # Approssima a quattro cifre decimali
        y = round(y, 4)  # Approssima a quattro cifre decimali
        results.extend([x, y, 0, cluster_num, cluster_num, cluster_letter])

# Nome del file in cui salvare i risultati
file_name = "risultatiD.txt"

# Scrivi i risultati nel file, separati da spazi
with open(file_name, 'w') as file:
    result_str = " ".join(map(str, results))
    file.write(f"{result_str}\n")

print(f"Results saved in {file_name}")



In [ ]:
# @title CHOOSER

# Nome del file in cui salvare i risultati
file_name = "coordinate_risultatiD.txt"

# Scrivi le coordinate approssimate a due cifre decimali nel file
with open(file_name, 'w') as file:
    for label in unique_labels:
        cluster_points = reduced_features[cluster_labels == label]
        for point in cluster_points:
            x, y = point[:2]  # Prendi solo le prime due coordinate PC1 e PC2
            x = round(x, 2)  # Approssima a due cifre decimali
            y = round(y, 2)  # Approssima a due cifre decimali
            file.write(f"append {x:.2f}:{y:.2f}, ")

print(f"Coordinates of results saved in {file_name}")


# Clustering 3D (Optional)

In [ ]:
# @title **Clustering 3D**
import plotly.express as px
import plotly.graph_objs as go
from sklearn.decomposition import PCA
import numpy as np


#Parametri
n_clusters = 4 # @param {type:"integer"}
init='k-means++'  # @param ["k-means++", "random"]
n_init=100 # @param {type:"integer"}
max_iter=10 # @param {type:"integer"}





scaled_features = np.load('scaled_features.npy')

# Crea il modello PCA con 3 componenti principali
pca = PCA(n_components=3)

# Applica la PCA alle feature scalate
reduced_features = pca.fit_transform(scaled_features)

# # Riscala i risultati dell'PCA tra 0 e 1
scaler = MinMaxScaler()
reduced_features = scaler.fit_transform(reduced_features)


# Esegui il clustering sulle feature ridotte
minibatch_kmeans = MiniBatchKMeans(n_clusters=n_clusters, init=init, n_init=n_init, max_iter=max_iter)
cluster_labels = minibatch_kmeans.fit_predict(reduced_features)

# # Calcola il Silhouette Score per valutare la bontà del clustering
# silhouette_avg = silhouette_score(reduced_features, cluster_labels)
# print(f"Silhouette Score: {silhouette_avg}")

# # Calcola l'inerzia (inertia) per valutare la compattazione dei cluster
# inertia = minibatch_kmeans.inertia_
# print(f"Inertia (Inerzia): {inertia}")

# Visualizza i risultati del clustering in un grafico 3D
fig = go.Figure()
unique_labels = np.unique(cluster_labels)
for label in unique_labels:
    cluster_points = reduced_features[cluster_labels == label]
    fig.add_trace(go.Scatter3d(x=cluster_points[:, 0], y=cluster_points[:, 1], z=cluster_points[:, 2], mode='markers', name=f'Cluster {label + 1}'))

fig.update_layout(scene=dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3'), title='Risultato del Clustering dei Loop Audio con MiniBatchKMeans e PCA (3D)')
fig.show()
